In [1]:
from data.dataset import DatasetLoader
dataset = DatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_transformed'
)

In [2]:
# Test batching
from preprocessing.batching_v2 import get_batch_graphs

get_batch_graphs(
    data=dataset[0],
    batch_size=100,
    recursive=True
)

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])
